In [9]:
import pandas as pd

In [13]:
lista_processos = ["53500.073244/2024-12", "53500.073223/2024-05", "53500.073219/2024-39", "53500.073206/2024-60", "53500.073204/2024-71"] 

In [7]:
caminho = input("Insira o caminho da planilha geral: ")
file_path = caminho.replace('"','').replace('\\','/')

In [22]:
# Carregar a planilha geral de processos
df = pd.read_excel(file_path, usecols=[0,2])

In [ ]:
# Iterar sobre cada processo
for processo in lista_processos:
    # Procurar o processo na primeira coluna
    resultado = df[df['Nº do Processo SEI'] == processo]  # Pega o processo na primeira coluna
    
    if not resultado.empty:
        # Pegar o valor da terceira coluna (retido)
        retido = resultado.iloc[0]['Retido'] 

        # Verificar se o valor é "Sim" ou "Não"
        if retido == 'Sim':
            print(f"Processo {processo}: Retido = Sim")
        elif retido == 'Não':
            print(f"Processo {processo}: Retido = Não")
        else:
            print(f"Processo {processo}: Valor inesperado = {retido}")
    else:
        print(f"Processo {processo} não encontrado")

In [1]:
#IMPORTS NECESSÁRIOS PARA O FUNCIONAMENTO DO CÓDIGO
import unidecode
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.common.by import By
import funcoes as fc

In [8]:
navegador = fc.abreChromeEdge()
while True:
    #INICIA JANELA
    try:
        fc.iniciaJanela(navegador)
    #CONDICAO DE ERRO PARA CASO O USUÁRIO ERRE O SEU LOGIN 
    except Exception:
        print('Ocorreu um erro, tente novamente.')
    #ENTRAR NA CAIXA DE PROCESSOS ATRIBUIDOS AO USUARIO
    if fc.check_element_exists(By.XPATH,'//*[@id="divFiltro"]/div[2]/a', navegador):
        navegador.find_element(By.XPATH,'//*[@id="divFiltro"]/div[2]/a').click()
        break
    #CASO NAO ENCONTRE O FILTRO DE PROCESSOS ELE INFOMA QUE NAO ENTROU NO SEI
    else:
        #APAGA O USUÁRIO
        navegador.find_element(By.XPATH,'//*[@id="txtUsuario"]').clear()
        print('Usuário ou senha incorretos. Digite novamente')

Usuário ou senha incorretos. Digite novamente


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\lukasa.estagio\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "c:\Users\lukasa.estagio\OneDrive - ANATEL\Área de Trabalho\Theomation_Rep\SEIAutomacao\funcoes.py", line 387, in <lambda>
    root.bind('<Return>', lambda event: preencher_campos(user_var, senha_var, navegador, root))
                                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lukasa.estagio\OneDrive - ANATEL\Área de Trabalho\Theomation_Rep\SEIAutomacao\funcoes.py", line 24, in preencher_campos
    navegador.find_element(By.XPATH,'//*[@id="txtUsuario"]').clear()
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lukasa.estagio\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 748, in find_element
    return se

In [ ]:
#//*[@id="txtUsuario"]